### INST326 OOP Project 03

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

### The Project
Everyone will do the same project this time. This is a group project, so you must work in your assigned groups. Include the link to your group's GitHub repository (one link per group). Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow all the requirements and the Notebook Instructions at the bottom of this notebook. Your grade may depend on it!

#### 1. A Scheduling Program>  My wife is responsible for scheduling caregivers for her 93 year-old mother. Currently she writes out the schedule on a monthly calendar and photocopies it for everyone. I want all of you to help me write a program to help her with scheduling. While this is a specific application, this program will be broadly useful and adaptable to any scheduling needs for small businesses, clubs, and more.

#### Requirements
>  Care is required 12 hours per day, 7 days a week. There are two shifts each day: 7:00 AM - 1:00 PM, and 1:00 PM to 7:00 PM. There are a total of 8 caregivers. Some are family members and some are paid. Each caregiver has their own availability for shifts that is generally the same from month to month, but there are exceptions for work, vacations, and other responsibilities. Your program should do the following:
> 1. Manage caregivers and their schedules. Attributes include: name, phone, email, pay rate, and hours.
> 2. Each caregiver should have their own availability schedule where they can indicate their availability for each shift. Availability categories are 'preferred', 'available' (default), and 'unavailable'.
> 3. Create a care schedule that covers AM and PM shifts and displays caregiver names on a calendar (see example). The schedule should accomodate caregivers' individual schedules and availability preferences. The python calendar module provides options for creating HTML calendars. Sample code for the HTML calendar is in the project folder.
> 4. Paid caregivers are paid weekly at $20/hr. Your program should calculate weekly pay based on assigned hours. Provide a separate pay report that lists weekly (gross: hours x rate) amounts to each caregiver, along with weekly and monthly totals. The report can be a text document, or presented in GUI or HTML format. 

#### Group Requirements
>  1. Your submitted project should follow OOP principles like abstraction, encapsulation, inheritance, and polymorphism as appropriate. Your program should use classes. 
>  2. Select a group leader who will host the group's project repository on their GitHub.
>  3. Create the group repository and add a main program document. See example.
>  4. Create branches off the main program for each group member, and assign part of the program to each member.
>  5. Each member should work on their branch.
>  6. When each member is finished, merge the branches back into the main program. You may use 'merge' or 'pull requests', your choice.
>  7. iterate and debug as necessary.

#### Working with HTML
> Since this is a course on python, not HTML, you are not expected to know HTML. Therefore, you may copy applicable portions of the sample code or use AI to write the HTML portions of your application. Ypu should write the main python code yourself.


#### What you need to turn in
>  This is a group project. There will be one submission per group. Your submission will be graded as a group.
>  1. Include your group number and the names of all group members in the signature block at the top of this notebook.
>  2. In the cell below, paste the link to your project repository. One link per group. The grader will review the activity and history provided by GitHub. To add a hyperlink to a Jupyter markdown cell, follow the instructions in the cell below.
>  3. Below the GitHub Repository Link cell is a code cell. Copy and paste your final program code into this cell.

#### GitHub Repository Link
> Example: [INST326_Fall2024/Projects/Project03](https://github.com/sdempwolf/INST326_Fall_2024/tree/main/Projects/Project03)
>
> Edit the link code below with your information, then run this cell. Test the link! It should take you to your GitHub project repository.
> [external link text](http://url_here)

In [1]:
from abc import ABC, abstractmethod
import calendar
import random

class Schedule:
    shifts = ["7:00AM - 1:00PM", "1:00PM - 7:00PM"]

    def __init__(self, year, month):
        self._num_days = calendar.monthrange(year, month)[1]
        self._schedule = {
            day: {shift: None for shift in Schedule.shifts}
            for day in range(1, self._num_days + 1)
        }

    @property
    def schedule(self):
        return self._schedule

    @property
    def num_days(self):
        return self._num_days

    def set_shift(self, caregiver_name, day, shift_num):
        if day in self._schedule and 0 <= shift_num < len(Schedule.shifts):
            shift = Schedule.shifts[shift_num]
            self._schedule[day][shift] = caregiver_name


class Caregiver:
    def __init__(self, name, phone, email, pay_rate):
        self.name = name
        self.phone = phone
        self.email = email
        self.pay_rate = pay_rate
        self.hours_worked = 0
        self.schedule = None  #availability schedule later 

    def set_schedule(self, year, month):
        self.schedule = AvailabilitySchedule(year, month)


class AvailabilitySchedule(Schedule):
    def __init__(self, year, month):
        super().__init__(year, month)
        for day in self._schedule:
            for shift in self._schedule[day]:
                self._schedule[day][shift] = "Available"

    def set_preferred(self, day, shift_num):
        if day in self._schedule and 0 <= shift_num < len(Schedule.shifts):
            shift = Schedule.shifts[shift_num]
            self._schedule[day][shift] = "Preferred"

    def set_unavailable(self, day, shift_num):
        if day in self._schedule and 0 <= shift_num < len(Schedule.shifts):
            shift = Schedule.shifts[shift_num]
            self._schedule[day][shift] = "Unavailable"


class CareSchedule(Schedule):
    def __init__(self, year, month):
        super().__init__(year, month)

    def assign_shift(self, caregiver, day, shift_num):
        if day in self.schedule and 0 <= shift_num < len(Schedule.shifts):
            shift = Schedule.shifts[shift_num]
            if caregiver.schedule.schedule[day][shift] in ("Available", "Preferred"):
                self.schedule[day][shift] = caregiver.name
                caregiver.hours_worked += 6  # Assuming each shift is 6 hours

    def generate_schedule(self, caregivers):
        for day in range(1, self.num_days + 1):
            for shift_num in range(len(Schedule.shifts)):
                if self.schedule[day][Schedule.shifts[shift_num]] is None:
                    for caregiver in caregivers:
                        if caregiver.schedule.schedule[day][Schedule.shifts[shift_num]] == "Preferred":
                            self.assign_shift(caregiver, day, shift_num)
                            break
                    else:
                        for caregiver in caregivers:
                            if caregiver.schedule.schedule[day][Schedule.shifts[shift_num]] == "Available":
                                self.assign_shift(caregiver, day, shift_num)
                                break


class HiredCaregivers:
    def __init__(self):
        self.caregivers = []

    def add_caregiver(self, caregiver):
        self.caregivers.append(caregiver)

    def remove_caregiver(self, caregiver_name):
        self.caregivers = [c for c in self.caregivers if c.name != caregiver_name]

    def report(self):
        print("Caregiver Pay Report:")
        total_weekly_pay = 0
        total_monthly_pay = 0

        for caregiver in self.caregivers:
            weekly_hours = min(caregiver.hours_worked, 40)  # Assuming max weekly hours
            monthly_hours = weekly_hours * 4
            weekly_pay = weekly_hours * caregiver.pay_rate
            monthly_pay = monthly_hours * caregiver.pay_rate

            total_weekly_pay += weekly_pay
            total_monthly_pay += monthly_pay

            print(f"{caregiver.name}:")
            print(f"  Weekly Hours: {weekly_hours}, Weekly Pay: ${weekly_pay:.2f}")
            print(f"  Monthly Hours: {monthly_hours}, Monthly Pay: ${monthly_pay:.2f}")

        print(f"Total Weekly Pay: ${total_weekly_pay:.2f}")
        print(f"Total Monthly Pay: ${total_monthly_pay:.2f}")


# Example usage
caregiver1 = Caregiver("Alice", "123-456-7890", "alice@example.com", 20)
caregiver2 = Caregiver("Bob", "987-654-3210", "bob@example.com", 18)

hired_caregivers = HiredCaregivers()
hired_caregivers.add_caregiver(caregiver1)
hired_caregivers.add_caregiver(caregiver2)

# Set availability schedules
caregiver1.set_schedule(2024, 11)
caregiver2.set_schedule(2024, 11)

# Create care schedule
care_schedule = CareSchedule(2024, 11)
care_schedule.generate_schedule(hired_caregivers.caregivers)

# Generate report
hired_caregivers.report()

Caregiver Pay Report:
Alice:
  Weekly Hours: 40, Weekly Pay: $800.00
  Monthly Hours: 160, Monthly Pay: $3200.00
Bob:
  Weekly Hours: 0, Weekly Pay: $0.00
  Monthly Hours: 0, Monthly Pay: $0.00
Total Weekly Pay: $800.00
Total Monthly Pay: $3200.00


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS

In [15]:
import calendar
from datetime import datetime

class Caregiver:
    def __init__(self, name, phone, email, hourly_rate):
        self.name = name
        self.phone = phone
        self.email = email
        self.hourly_rate = hourly_rate
        self.schedule = {}
        self.total_hours = 0  # Add an attribute to track total hours worked

    def set_schedule(self, year, month, days_schedule):
        self.schedule[f"{year}-{month}"] = days_schedule

    def calculate_weekly_pay(self):
        # Assuming each shift (morning or evening) is 1 hour
        self.total_hours = 0
        for days in self.schedule.values():
            for day_schedule in days:
                # Count the available shifts as 1 hour per shift
                self.total_hours += day_schedule.count("available")
        return self.total_hours * self.hourly_rate


class CareSchedule:
    def __init__(self, year, month):
        self.year = year
        self.month = month
        self.schedule = {}

    def generate_schedule(self, caregivers):
        print(f"Generating care schedule for {self.year}-{self.month}...")
        # Initialize the daily schedule (one caregiver per day)
        daily_schedule = {}
        days_in_month = calendar.monthrange(self.year, self.month)[1]

        for day in range(1, days_in_month + 1):
            assigned_caregiver_morning = None
            assigned_caregiver_evening = None
            for caregiver in caregivers:
                morning, evening = caregiver.schedule[f"{self.year}-{self.month}"][day-1]  # 0-indexed
                if morning == "available" and not assigned_caregiver_morning:
                    assigned_caregiver_morning = caregiver.name
                if evening == "available" and not assigned_caregiver_evening:
                    assigned_caregiver_evening = caregiver.name

            daily_schedule[f"Day {day}"] = {
                "Morning": assigned_caregiver_morning if assigned_caregiver_morning else "No caregiver available",
                "Evening": assigned_caregiver_evening if assigned_caregiver_evening else "No caregiver available"
            }

        self.schedule = daily_schedule

    def generate_html(self, caregivers):
        # Start HTML structure
        html_content = f"""
        <html>
        <head>
            <title>Caregiving Schedule - {self.year}-{self.month}</title>
            <style>
                table {{
                    width: 100%;
                    border-collapse: collapse;
                }}
                th, td {{
                    border: 1px solid #ddd;
                    padding: 8px;
                    text-align: center;
                    height: 100px;
                    width: 14%;
                }}
                th {{
                    background-color: #f2f2f2;
                }}
                .calendar {{
                    display: grid;
                    grid-template-columns: repeat(7, 1fr);
                    gap: 5px;
                    margin-top: 20px;
                }}
                .day {{
                    border: 1px solid #ddd;
                    padding: 5px;
                    text-align: center;
                }}
                .day-header {{
                    background-color: #f2f2f2;
                    font-weight: bold;
                }}
                .shift-info {{
                    margin-top: 5px;
                    font-size: 12px;
                }}
                .pay-info {{
                    margin-top: 10px;
                    font-size: 14px;
                    color: green;
                }}
            </style>
        </head>
        <body>
            <h2>Caregiving Schedule for {self.year}-{self.month}</h2>
            <div class="calendar">
                <div class="day-header">Sun</div>
                <div class="day-header">Mon</div>
                <div class="day-header">Tue</div>
                <div class="day-header">Wed</div>
                <div class="day-header">Thu</div>
                <div class="day-header">Fri</div>
                <div class="day-header">Sat</div>
        """

        # Get the first weekday and number of days in the month
        first_weekday, num_days = calendar.monthrange(self.year, self.month)
        
        # Generate the calendar
        for i in range(first_weekday):
            html_content += "<div class='day'></div>"  # Empty cells before the first day
        
        for day in range(1, num_days + 1):
            caregiver_morning = self.schedule[f"Day {day}"]["Morning"]
            caregiver_evening = self.schedule[f"Day {day}"]["Evening"]
            html_content += f"""
            <div class="day">
                <div>{day}</div>
                <div class="shift-info">Morning: {caregiver_morning}</div>
                <div class="shift-info">Evening: {caregiver_evening}</div>
            </div>
            """
            if (first_weekday + day) % 7 == 0:  # Start a new row after Saturday
                html_content += "</div><div class='calendar'>"

        # Close the HTML tags
        html_content += "</div>"

        # Add caregivers' weekly pay to the HTML
        html_content += "<h3>Weekly Pay</h3><table><tr><th>Caregiver</th><th>Weekly Pay</th></tr>"
        for caregiver in caregivers:
            weekly_pay = caregiver.calculate_weekly_pay()
            html_content += f"<tr><td>{caregiver.name}</td><td>${weekly_pay}</td></tr>"
        html_content += "</table>"

        html_content += "</body></html>"

        # Save the output to an HTML file
        file_name = f"caregiving_schedule_{self.year}_{self.month}.html"
        with open(file_name, "w") as file:
            file.write(html_content)

        print(f"HTML calendar for {self.year}-{self.month} has been generated.")

# Example Caregiver setup
caregiver1 = Caregiver("Alice", "123-456-7890", "alice@example.com", 0)
caregiver2 = Caregiver("Bob", "987-654-3210", "bob@example.com", 0)
caregiver3 = Caregiver("Charlie", "321-654-9870", "charlie@example.com", 0)
caregiver4 = Caregiver("David", "555-555-5555", "david@example.com", 0) #represents family members 
caregiver5 = Caregiver("Eve", "222-333-4444", "eve@example.com", 20) #represents $20/hr wage
caregiver6 = Caregiver("Frank", "444-555-6666", "frank@example.com", 20)
caregiver7 = Caregiver("Grace", "777-888-9999", "grace@example.com", 20)
caregiver8 = Caregiver("Hannah", "666-777-8888", "hannah@example.com", 20)

# Add caregivers to the schedule
caregiver1.set_schedule(2024, 11, [("available", "not available")] * 30)
caregiver2.set_schedule(2024, 11, [("not available", "available")] * 30)
caregiver3.set_schedule(2024, 11, [("available", "available")] * 30)
caregiver4.set_schedule(2024, 11, [("not available", "available")] * 30)
caregiver5.set_schedule(2024, 11, [("available", "available")] * 30)
caregiver6.set_schedule(2024, 11, [("not available", "not available")] * 30)
caregiver7.set_schedule(2024, 11, [("available", "not available")] * 30)
caregiver8.set_schedule(2024, 11, [("not available", "not available")] * 30)

hired_caregivers = [caregiver1, caregiver2, caregiver3, caregiver4, caregiver5, caregiver6, caregiver7, caregiver8]

# Create the care schedule and generate the HTML output
care_schedule = CareSchedule(2024, 11)
care_schedule.generate_schedule(hired_caregivers)
care_schedule.generate_html(hired_caregivers)

Generating care schedule for 2024-11...
HTML calendar for 2024-11 has been generated.
